In [6]:
from numpy import ones_like, mgrid, sqrt

from pysph.base.utils import get_particle_array
from pysph.solver.application import Application
from pysph.sph.scheme import WCSPHScheme

In [7]:
class EllipticalDrop(Application):
    # ...
    def create_particles(self):
        """Create the circular patch of fluid."""
        dx = self.dx
        hdx = self.hdx
        ro = self.ro
        name = 'fluid'
        x, y = mgrid[-1.05:1.05+1e-4:dx, -1.05:1.05+1e-4:dx]
        x = x.ravel()
        y = y.ravel()

        m = ones_like(x)*dx*dx
        h = ones_like(x)*hdx*dx
        rho = ones_like(x) * ro
        u = -100*x
        v = 100*y

        # remove particles outside the circle
        indices = []
        for i in range(len(x)):
            if sqrt(x[i]*x[i] + y[i]*y[i]) - 1 > 1e-10:
                indices.append(i)

        pa = get_particle_array(x=x, y=y, m=m, rho=rho, h=h, u=u, v=v,
                                name=name)
        pa.remove_particles(indices)

        print("Elliptical drop :: %d particles"
              % (pa.get_number_of_particles()))

        self.scheme.setup_properties([pa])
        return [pa]

In [3]:
def create_equations(self):
    equations = [
        Group(equations=[
            TaitEOS(dest='fluid', sources=None, rho0=self.ro,
                    c0=self.co, gamma=7.0),
        ], real=False),

        Group(equations=[
            ContinuityEquation(dest='fluid',  sources=['fluid',]),

            MomentumEquation(dest='fluid', sources=['fluid'],
                             alpha=self.alpha, beta=0.0, c0=self.co),

            XSPHCorrection(dest='fluid', sources=['fluid']),

        ]),
    ]
    return equations


In [9]:
!pysph run elliptical_drop_simple --tf=0.005

Running example pysph.examples.elliptical_drop_simple.

Information for example: pysph.examples.elliptical_drop_simple
Evolution of a circular patch of incompressible fluid. (30 seconds)

This is the simplest implementation using existing schemes.

See J. J. Monaghan "Simulating Free Surface Flows with SPH", JCP, 1994, 100, pp
399 - 406
Elliptical drop :: 5025 particles
Generating output in /usr/src/app/pysph/elliptical_drop_simple_output
Compiling code at: /root/.pysph/source/py3.9-linux-x86_64/m_30bb3da0068142637bd63ff68cf26310.pyx
No of particles:
----------------------------------------------------------------------
  fluid: 5025
----------------------------------------------------------------------
Setup took: 10.62624 secs
100%
Run took: 11.79780 secs
